In [ ]:
!unzip artinsight_poc_mockai.zip -d artinsight_poc



Archive:  artinsight_poc_mockai.zip
  inflating: artinsight_poc/README.md  
  inflating: artinsight_poc/LICENSE  
  inflating: artinsight_poc/src/monitoring.py  
  inflating: artinsight_poc/src/analysis.py  
  inflating: artinsight_poc/src/scoring.py  
  inflating: artinsight_poc/src/analytics.py  
  inflating: artinsight_poc/src/utils.py  
  inflating: artinsight_poc/src/dashboard.py  
  inflating: artinsight_poc/docs/architecture.md  
  inflating: artinsight_poc/docs/algorithms.md  
  inflating: artinsight_poc/notebooks/poc_colab_instructions.md  
  inflating: artinsight_poc/notebooks/poc_starter.py  


In [ ]:
!pip install streamlit matplotlib pandas pyngrok


In [ ]:
import os
import json
import pandas as pd
from dash import Dash, dcc, html, Input, Output
from pyngrok import ngrok
import nest_asyncio
import threading
import subprocess
import signal
import time

# Apply asyncio patch for Colab
nest_asyncio.apply()

# ✅ Set working directory (adjust if needed)
os.chdir('/content/artinsight_poc')

# 1️⃣ Ensure sample data exists
sample_file = "sample_opportunities.json"

if not os.path.exists(sample_file):
    # If you have your utils/scoring modules, generate sample data
    from src import monitoring, analysis, scoring, utils

    posts = monitoring.fetch_reddit(mock=True, count=5)
    posts += monitoring.fetch_pinterest(mock=True, count=5)

    scored = []
    for p in posts:
        p['score'] = scoring.score_opportunity(p)
        p['suggestion'] = analysis.generate_suggestion(p)
        p['approved'] = False
        scored.append(p)

    utils.save_sample(scored, path=sample_file)
    print(f"✅ Generated {sample_file} with {len(scored)} items")
else:
    print(f"✅ {sample_file} already exists")

# 2️⃣ Load sample data
with open(sample_file) as f:
    data = json.load(f)
df = pd.DataFrame(data)

# 3️⃣ Kill old ngrok tunnels
ngrok.kill()
print("✅ Old ngrok tunnels stopped")

# 4️⃣ Kill any processes using port 8050
try:
    result = subprocess.check_output("lsof -t -i:8050", shell=True).decode().split()
    for pid in result:
        try:
            os.kill(int(pid), signal.SIGTERM)
        except ProcessLookupError:
            pass
    if result:
        print("✅ Killed old processes on port 8050")
except subprocess.CalledProcessError:
    print("✅ No processes found on port 8050")

# 5️⃣ Create Dash app
app = Dash(__name__)
app.layout = html.Div([
    html.H1("ArtInsight POC Dashboard"),
    html.Label("Select Post:"),
    dcc.Dropdown(
        id="select-post",
        options=[{"label": p["text"], "value": i} for i, p in df.iterrows()],
        value=0
    ),
    html.Br(),
    html.Div(id="post-details")
])

@app.callback(
    Output("post-details", "children"),
    Input("select-post", "value")
)
def show_post_details(idx):
    post = df.iloc[idx]
    return html.Div([
        html.P(f"Title: {post['text']}"),
        html.P(f"Score: {post['score']}"),
        html.P(f"Suggestion: {post['suggestion']}"),
        html.P(f"Approved: {post['approved']}")
    ])

# 6️⃣ Run Dash app in a separate thread
def run_dash_app():
    app.run(port=8050, debug=False)

threading.Thread(target=run_dash_app, daemon=True).start()

# 7️⃣ Wait for server to start
time.sleep(5)

# 8️⃣ Set ngrok auth token
ngrok.set_auth_token("36OLnCkKP5EJca0oqrVCDpVnOQ9_nrqWqnX6Vh2rzjJrnE6n")  # replace with your token

# 9️⃣ Open ngrok tunnel with pooling
public_url = ngrok.connect(8050, options={"pooling_enabled": True})
print(f"🎉 Dash app is live at: {public_url}")


✅ sample_opportunities.json already exists
✅ Old ngrok tunnels stopped
✅ No processes found on port 8050


<IPython.core.display.Javascript object>

PyngrokNgrokHTTPError: ngrok client exception, API returned 400: {"error_code":102,"status_code":400,"msg":"invalid tunnel configuration","details":{"err":"yaml: unmarshal errors:\n  line 1: field options not found in type config.HTTPv2Tunnel"}}
